In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.functional as F
from tqdm.auto import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sklearn.model_selection import train_test_split

In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
data = pd.read_csv("../data/raw_data/rawdat/AFG/quadruple.txt", sep='\t', names = ["source","relation","target","date"])
data.head()

,source,relation,target,date
0,Armed Gang (Afghanistan),Carry out suicide bombing,United States,2010-01-01
1,Central Intelligence Agency,Make statement,Taliban,2010-01-01
2,Taliban,Make statement,Attacker (Afghanistan),2010-01-01
3,Citizen (Afghanistan),Demonstrate or rally,Unspecified Actor,2010-01-01
4,Armed Gang (Afghanistan),Carry out suicide bombing,Central Intelligence Agency,2010-01-01


In [4]:
# data = pd.read_csv("../data/raw_data/rawdat/AFG/quadruple.txt", sep='\t', names = ["source","relation","target","date"])
# data.head()

In [5]:
text =pd.read_csv("../data/raw_data/rawdat/AFG/text.txt", sep='\t', names = ["text"])
text.head()

,text
0,"According to a foreign news agency, a suicide ..."
1,The CIA said a Taliban bomber on Wednesday man...
2,The Taliban claimed responsibility for the att...
3,The case in Kunar has already prompted Afghans...
4,The CIA base attacked by a suicide bomber in A...


In [6]:
data['source'] = data['source'].str.strip()
data['relation'] = data['relation'].str.strip()
data['target'] = data['target'].str.strip()
text['text'] = text['text'].str.strip()

In [7]:
result = pd.concat([data,text],axis=1)
result.head()

,source,relation,target,date,text
0,Armed Gang (Afghanistan),Carry out suicide bombing,United States,2010-01-01,"According to a foreign news agency, a suicide ..."
1,Central Intelligence Agency,Make statement,Taliban,2010-01-01,The CIA said a Taliban bomber on Wednesday man...
2,Taliban,Make statement,Attacker (Afghanistan),2010-01-01,The Taliban claimed responsibility for the att...
3,Citizen (Afghanistan),Demonstrate or rally,Unspecified Actor,2010-01-01,The case in Kunar has already prompted Afghans...
4,Armed Gang (Afghanistan),Carry out suicide bombing,Central Intelligence Agency,2010-01-01,The CIA base attacked by a suicide bomber in A...


In [8]:
result['text'][0]

'According to a foreign news agency, a suicide bomber attacked the US CIA center in Khost, Afghanistan, killing eight CIA officials and wounding six others.'

In [9]:
unique_result = result.drop_duplicates()
# unique_result = result.drop_duplicates(subset=['text'],keep='last')

In [10]:
# len(result)

In [11]:
# len(unique_result)

In [12]:
# groupby_df = pd.DataFrame(unique_result.groupby(["source","date"])[['target','text','relation']].agg(lambda x: '///'.join(x)).reset_index())

In [13]:
# len(groupby_df)

In [14]:
# groupby_df.head()

In [15]:
# for i in range(200):
#     print(groupby_df['source'][i],"\t",groupby_df['relation'][i],"\t",groupby_df['target'][i],"\n", groupby_df['text'][i],"\n")

In [16]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

C:\Users\desai\anaconda3\envs\cuda_test\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Tried In-context learning, gives answers close to what we want, but hallucinates sometimes. There is a limit to number of input tokens that can be provided.

In [17]:
# print(f"Unique Sources: {len(result['source'].unique())} \nUnique Targets: {len(result['target'].unique())}\nUnique Relations: {len(result['relation'].unique())} \nUnique Texts: {len(result['text'].unique())}")

In [18]:
# input_text = "Given an event text, extract the source, target and relation between them: For e.g.\n"
# # for t in range(len(result)):
# for t in range(len(result)):
#     if len(input_text.split())>=512:
#         input_text += f"Given Text: {result['text'][t]}"
#         print(f"Prediction text :{result['text'][t]}\n Extracted information -> Source: {result['source'][t]}, Target: {result['target'][t]}, Relation: {result['relation'][t]}\n")
#         break
#     input_text += f"Given text:{result['text'][t]}\n Extracted information -> Source:{result['source'][t]}, Target:{result['target'][t]}, Relation:{result['relation'][t]}\n"

# input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

In [19]:
# outputs = model.generate(input_ids,max_new_tokens=1000)
# print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Tried Chain-of-Thought Prompting. Didn't work well as the reasoning didn't make much sense.

In [20]:
# cot_input = f"Given a text: 'In western Farah province, which like Badghis is experiencing escalating violence as Taliban influence spreads to previously peaceful areas, three militants were killed by police in a shootout late Thursday, police said', we extract the source:'Police (Afghanistan)', target:'Police (Afghanistan)' and relation:' Use conventional military force'. Explain the reasoning as to how the source, target and relation are extracted based on the text."
# cot_input_ids = tokenizer(cot_input, return_tensors="pt").input_ids.to("cuda")

In [21]:
# cot_outputs = model.generate(cot_input_ids,max_new_tokens=1000)
# print(tokenizer.decode(cot_outputs[0],skip_special_tokens=True))

### Training a supervised model using FLAN-T5 for multiclass classification of event type

In [22]:
# def generate_training_data(df: pd.DataFrame, ids: np.ndarray, masks: np.ndarray, tokenizer):
#     for i,text in tqdm(enumerate(df['text'])):
#         tokens = tokenizer.encode_plus(
#             text,
#             max_length = 256,
#             truncation = True,
#             padding = 'max_length',
#             add_special_tokens = True,
#             return_tensors = 'pt'          
#         )
#         ids[i, :] = tokens.input_ids
#         masks[i, :] = tokens.attention_mask
#     return ids, masks        

In [23]:
# X_input_ids = np.zeros((len(unique_result),256))
# X_attn_masks = np.zeros((len(unique_result),256))

In [24]:
# X_input_ids, X_attn_masks = generate_training_data(unique_result, X_input_ids, X_attn_masks, tokenizer)

In [25]:
# len(unique_result['relation'].unique())

In [26]:
relation_ids = pd.read_csv('../data/raw_data/rawdat/AFG/relation2id.txt', sep='\t', names=['relation','id'])
relation_ids.head()

,relation,id
0,Carry out suicide bombing,0
1,Make statement,1
2,Demonstrate or rally,2
3,Meet at a 'third' location,3
4,Employ aerial weapons,4


In [27]:
relation_id_maps = pd.Series(relation_ids.id.values, index=relation_ids.relation).to_dict()
relation_id_maps

{'Carry out suicide bombing': 0,
 'Make statement': 1,
 'Demonstrate or rally': 2,
 "Meet at a 'third' location": 3,
 'Employ aerial weapons': 4,
 'Acknowledge or claim responsibility': 5,
 'Engage in negotiation': 6,
 'Make an appeal or request': 7,
 'Use conventional military force': 8,
 'Consult': 9,
 'Threaten with military force': 10,
 'Mobilize or increase armed forces': 11,
 'Abduct, hijack, or take hostage': 12,
 'Use unconventional violence': 13,
 'Make a visit': 14,
 'Host a visit': 15,
 'Appeal for diplomatic cooperation (such as policy support)': 16,
 'Provide aid': 17,
 'Conduct suicide, car, or other non-military bombing': 18,
 'Investigate': 19,
 'Accuse': 20,
 'Criticize or denounce': 21,
 'Reject': 22,
 'Arrest, detain, or charge with legal action': 23,
 'Praise or endorse': 24,
 'Demand': 25,
 'fight with artillery and tanks': 26,
 'Deny responsibility': 27,
 'Use as human shield': 28,
 'Accuse of human rights abuses': 29,
 'fight with small arms and light weapons': 3

In [28]:
unique_result['relation_to_id'] = unique_result['relation'].map(relation_id_maps)

C:\Users\desai\AppData\Local\Temp\ipykernel_11272\28446876.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_result['relation_to_id'] = unique_result['relation'].map(relation_id_maps)


In [29]:
unique_result.head()

,source,relation,target,date,text,relation_to_id
0,Armed Gang (Afghanistan),Carry out suicide bombing,United States,2010-01-01,"According to a foreign news agency, a suicide ...",0
1,Central Intelligence Agency,Make statement,Taliban,2010-01-01,The CIA said a Taliban bomber on Wednesday man...,1
2,Taliban,Make statement,Attacker (Afghanistan),2010-01-01,The Taliban claimed responsibility for the att...,1
3,Citizen (Afghanistan),Demonstrate or rally,Unspecified Actor,2010-01-01,The case in Kunar has already prompted Afghans...,2
4,Armed Gang (Afghanistan),Carry out suicide bombing,Central Intelligence Agency,2010-01-01,The CIA base attacked by a suicide bomber in A...,0


In [30]:
# X_input_ids

In [31]:
# labels = np.zeros((len(unique_result), len(relation_ids)))
# labels.shape

In [32]:
# labels[np.arange(len(unique_result)), unique_result['relation_to_id'].values] = 1

In [33]:
# labels

In [34]:
class EventDataset(Dataset):
    def __init__(self, dataframe, tokenizer, source_len, source_text, target_label):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
#         self.target_len = target_len
        self.source_text = dataframe[source_text]
        self.target_label = dataframe[target_label]
    def __len__(self):
        return len(self.target_label)
    
    def __getitem__(self,index):
        source_text = str(self.source_text[index])
        target_label = int(self.target_label[index])
        # print("target_label: ",target_label)
        
        lm_labels = np.zeros((1, len(relation_ids)))
        lm_labels[np.arange(1), target_label] = 1
        
        source = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        # print("source_text: ",source_text)
        # print("source: ",source)
        # target = self.tokenizer.batch_encode_plus(
        #     [lm_labels],
        #     # max_length=len(self.target_label.unique()),
        #     # pad_to_max_length=True,
        #     # truncation=True,
        #     # padding="max_length",
        #     return_tensors="pt",
        # )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
#         target_ids = target["input_ids"].squeeze()
#         target_mask = target["attention_mask"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
#             "target_ids": target_ids.to(dtype=torch.long),
#             "target_ids_y": target_ids.to(dtype=torch.long),
            # "labels":lm_labels.to(dtype=torch.long)
            "labels": torch.tensor(lm_labels, dtype=torch.long),
        }

In [35]:
labels_ex = np.zeros((1, 10))
labels_ex[np.arange(1),5] = 1
labels_ex

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [36]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    """
    Function to be called for training with the parameters passed from main function
    """

    model.train()
    for _, data in enumerate(loader, 0):
        y = data["labels"].to(device, dtype=torch.long)
        print("y: ",y)
        # y_ids = y[:, :-1].contiguous()
        # lm_labels = y[:, 1:].clone().detach()
        # lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100

        # lm_labels = np.zeros((len(unique_result), len(relation_ids)))
        # lm_labels = np.zeros((1, len(relation_ids)))
        # lm_labels[np.arange(1), unique_result['relation_to_id'].values] = 1
        
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)
        print(len(ids), len(mask), len(y))
        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            # decoder_input_ids=y_ids,
            # labels=lm_labels,
            labels=y
        )
        loss = outputs[0]

        if _ % 10 == 0:
            training_logger.add_row(str(epoch), str(_), str(loss))
            console.print(training_logger)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [37]:
def validate(epoch, tokenizer, model, device, loader):
  """
  Function to evaluate model for predictions
  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['labels'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals


In [38]:
def T5Trainer(
    dataframe, source_text,tokenizer, target_text, model, model_params, output_dir="./outputs/"
):

    """
    T5 trainer

    """

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True

    # logging
    print(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    # tokenzier for encoding the text
#     tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])
    tokenizer = tokenizer

    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"], device_map='auto')
    # model = model
    model = model.to(device)

    # logging
    print(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    dataframe = dataframe[[source_text, target_text]]
    print(dataframe.head(2))

    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest for validation.
    train_size = 0.8
    train_dataset = dataframe.sample(frac=train_size, random_state=model_params["SEED"])
    val_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    print(f"FULL Dataset: {dataframe.shape}")
    print(f"TRAIN Dataset: {train_dataset.shape}")
    print(f"TEST Dataset: {val_dataset.shape}\n")

    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = EventDataset(
        train_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    val_set = EventDataset(
        val_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    # Defining the parameters for creation of dataloaders
    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }

    val_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)

    # Defining the optimizer that will be used to tune the weights of the network in the training session.
    optimizer = torch.optim.Adam(
        params=model.parameters(), lr=model_params["LEARNING_RATE"]
    )

    # Training loop
    print(f"[Initiating Fine Tuning]...\n")

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train(epoch, tokenizer, model, device, training_loader, optimizer)

    print(f"[Saving Model]...\n")
    # Saving the model after training
    path = os.path.join(output_dir, "model_files")
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)

    # evaluating test dataset
    print(f"[Initiating Validation]...\n")
    for epoch in range(model_params["VAL_EPOCHS"]):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({"Generated Text": predictions, "Actual Text": actuals})
        final_df.to_csv(os.path.join(output_dir, "predictions.csv"))

#     console.save_text(os.path.join(output_dir, "logs.txt"))

    print(f"[Validation Completed.]\n")
    print(
        f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n"""
    )
    print(
        f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n"""
    )
    print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [41]:

# let's define model parameters specific to T5
model_params = {
    "MODEL": "t5-large",  # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE": 8,  # training batch size
    "VALID_BATCH_SIZE": 8,  # validation batch size
    "TRAIN_EPOCHS": 3,  # number of training epochs
    "VAL_EPOCHS": 1,  # number of validation epochs
    "LEARNING_RATE": 1e-4,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 50,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
}

In [42]:
# let's get a news summary dataset
# dataframe has 2 columns: 
#   - text: long article content
#   - headlines: one line summary of news
# path = "https://raw.githubusercontent.com/Shivanandroy/T5-Finetuning-PyTorch/main/data/news_summary.csv"

# df = pd.read_csv(path)


# # T5 accepts prefix of the task to be performed:
# # Since we are summarizing, let's add summarize to source text as a prefix
# df["text"] = "summarize: " + df["text"]

T5Trainer(
    dataframe=unique_result,
    source_text="text",
    tokenizer = tokenizer,
    target_text="relation_to_id",
    model_params=model_params,
    model = model,
    output_dir="../notebooks and .py/outputs",
)


[Model]: Loading t5-large...

[Data]: Reading data...

                                                text  relation_to_id
0  According to a foreign news agency, a suicide ...               0
1  The CIA said a Taliban bomber on Wednesday man...               1
FULL Dataset: (254985, 2)
TRAIN Dataset: (203988, 2)
TEST Dataset: (50997, 2)

[Initiating Fine Tuning]...

y:  tensor([[[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 1, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 1, 0,  ..., 0, 0, 0]]], device='cuda:0')
8 8 8


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 

In [ ]:
dataset = EventDataset(X_input_ids,X_attn_masks,labels)

In [ ]:
dataloader = DataLoader(dataset, shuffle=True)

In [ ]:
samples = next(iter(dataloader))
samples

In [ ]:
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)